In [13]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk

In [14]:
# 감성분석 모델 불러오기
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-FinBert-SC')
model = AutoModelForSequenceClassification.from_pretrained('snunlp/KR-FinBert-SC')

senti_classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1)
    Python  3.10.11 (you have 3.10.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [21]:
senti_classifier(['과자'])

[{'label': 'neutral', 'score': 0.9997841715812683}]

In [2]:
text = pd.read_csv('../../뉴스 크롤링/관련주 크롤링/루닛.csv')

In [3]:
text.head()

,date,title,content
0,2023-06-30,[주식 초고수는 지금] '주가 올 500% 급등' 루닛 순매수 1위,"[\n하나기술, 에코프로 등 2차전지주도 순매수 상위권\n\n\n\n[서울경제] 미..."
1,2023-06-30,"휴대용 엑스레이 기업 오톰, 잇단 M&amp;A 러브콜",[\n\t\t\t[이데일리 석지헌 기자] 휴대 가능한 포터블 엑스레이를 제조 업체 ...
2,2023-06-30,성일하이텍·루닛 등... 내달 2.5억주 의무보유등록 해제,[\n\t\t\t30일 한국예탁결제원은 내달 41개사 2억5233만주의 의무보유등록...
3,2023-06-30,"외인·기관 매도세에…국내 증시, 약보합 출발","[\n코스피·코스닥, 동반 하락 출발외국인 투자자와 기관의 매도세에 국내 증시가 약..."
4,2023-06-30,개인투자자 순매수 톱10 중 9종목이 ‘마이너스’,[\n대부분 포털·2차전지 종목유일한 상승주는 AI업체 ‘루닛’\n\n\n\n/일러...


In [5]:
list_text = text['content'].tolist()

In [8]:
# list_text

In [6]:
sentences = []
for item in list_text:
    sentences.extend(item.split('. '))

In [8]:
sentences

['[\n하나기술, 에코프로 등 2차전지주도 순매수 상위권\n\n\n\n[서울경제] 미래에셋증권에서 거래하는 고수익 투자자들이 30일 오전 가장 많이 순매수한 종목은 루닛(328130)으로 나타났다',
 '하나기술, 에코프로, 포스코퓨처엠, 오픈놀 등도 순위에 올랐다.이날 미래에셋엠클럽에 따르면 미래에셋증권 주식 거래 고객 중 최근 1개월 간 투자수익률 상위 1%에 해당하는 ‘주식 초고수’들이 오전 11시까지 가장 많이 사들인 종목은 루닛으로 집계됐다',
 '지난 19일부터 28일까지 7거래일 연속 상승세를 기록하다가 이날 장 초반 주가가 9% 넘게 빠지며 조정을 받자 매수세가 몰리는 것으로 분석된다',
 '7연속 상승 국면에 접어들기 직전인 지난 16일부터 27일까지 상승률은 무려 79.41%에 달한다.루닛은 의료 인공지능(AI) 기업으로, 챗GPT 열풍 등에 힘입어 올해 들어서만(12월 29일~6월 27일) 주가가 499.66% 급등했다',
 "지난 26일 일본 후지필름과 개발해 판매하는 'CXR-AID'가 일본 건강보험 급여 가산 대상으로 공식 인증받았다는 소식도 강력한 호재로 작용했다.2위는 2차전지 장비 기업인 하나기술이다",
 '최근 대규모 수주 소식을 잇달아 전하자 매수세가 몰리는 것으로 분석된다',
 '하나기술은 지난 27일 1724억 원 규모의 2차전지 조립, 화성라인 턴키 공급 수주 소식을 알렸다',
 '판매공급 지역은 아시아로, 최근 3년간 계약 이행 여부 내역이 없는 신규 고객사와의 계약이다',
 '심원용 신한투자증권 연구원은 “단일 수주 계약 공시로는 최대 규모”라며 “발주금액을 고려하면 고객사는 이미 양산성이 검증된 배터리 플레이어일 가능성이 높다”고 밝혔다.심 연구원은 하나기술이 이번 수주로 실적 가시성을 확보한 것은 물론, 단일 고객사 리스크에서도 벗어날 것으로 전망했다',
 '그는 “이번 수주로 수주 잔고는 3800억원 수준까지 증가한 것으로 추정한다”며 “현재 1, 2공장 합산 생산능력(캐파)는 연 매출액 7000억 원 수준으로

In [10]:
# CSV 파일 읽기
data = pd.read_csv('../../뉴스 크롤링/관련주 크롤링/루닛.csv')

# 뉴스 기사 추출
news_articles = data['content'].tolist()  # 열이름은 실제로 있는 열의 이름으로 바꿔주세요

numbered_articles = []  # 번호가 부여된 기사들을 저장할 리스트

for article in news_articles:
    sentences = article.split('. ')  # 마침표와 공백으로 문장을 분리
    num_sentences = len(sentences)  # 문장 개수

    # 문장마다 번호 부여 및 리스트에 추가
    numbered_sentences = [f"{i+1}. {sentence}" for i, sentence in enumerate(sentences)]
    numbered_articles.append(numbered_sentences)

# 결과 출력
for numbered_article in numbered_articles:
    for sentence in numbered_article:
        print(sentence)

1. [
하나기술, 에코프로 등 2차전지주도 순매수 상위권



[서울경제] 미래에셋증권에서 거래하는 고수익 투자자들이 30일 오전 가장 많이 순매수한 종목은 루닛(328130)으로 나타났다
2. 하나기술, 에코프로, 포스코퓨처엠, 오픈놀 등도 순위에 올랐다.이날 미래에셋엠클럽에 따르면 미래에셋증권 주식 거래 고객 중 최근 1개월 간 투자수익률 상위 1%에 해당하는 ‘주식 초고수’들이 오전 11시까지 가장 많이 사들인 종목은 루닛으로 집계됐다
3. 지난 19일부터 28일까지 7거래일 연속 상승세를 기록하다가 이날 장 초반 주가가 9% 넘게 빠지며 조정을 받자 매수세가 몰리는 것으로 분석된다
4. 7연속 상승 국면에 접어들기 직전인 지난 16일부터 27일까지 상승률은 무려 79.41%에 달한다.루닛은 의료 인공지능(AI) 기업으로, 챗GPT 열풍 등에 힘입어 올해 들어서만(12월 29일~6월 27일) 주가가 499.66% 급등했다
5. 지난 26일 일본 후지필름과 개발해 판매하는 'CXR-AID'가 일본 건강보험 급여 가산 대상으로 공식 인증받았다는 소식도 강력한 호재로 작용했다.2위는 2차전지 장비 기업인 하나기술이다
6. 최근 대규모 수주 소식을 잇달아 전하자 매수세가 몰리는 것으로 분석된다
7. 하나기술은 지난 27일 1724억 원 규모의 2차전지 조립, 화성라인 턴키 공급 수주 소식을 알렸다
8. 판매공급 지역은 아시아로, 최근 3년간 계약 이행 여부 내역이 없는 신규 고객사와의 계약이다
9. 심원용 신한투자증권 연구원은 “단일 수주 계약 공시로는 최대 규모”라며 “발주금액을 고려하면 고객사는 이미 양산성이 검증된 배터리 플레이어일 가능성이 높다”고 밝혔다.심 연구원은 하나기술이 이번 수주로 실적 가시성을 확보한 것은 물론, 단일 고객사 리스크에서도 벗어날 것으로 전망했다
10. 그는 “이번 수주로 수주 잔고는 3800억원 수준까지 증가한 것으로 추정한다”며 “현재 1, 2공장 합산 생산능력(캐파)는 연 매출액 7000억 원 수준으로 대응 역량도 충분하다

In [15]:
# CSV 파일 읽기
data = pd.read_csv('../../뉴스 크롤링/관련주 크롤링/루닛.csv')

# 뉴스 기사 추출
news_articles = data['content'].tolist()  # 열이름은 실제로 있는 열의 이름으로 바꿔주세요

numbered_articles = []  # 번호가 부여된 기사들을 저장할 리스트

for article in news_articles:
    sentences = article.split('. ')  # 마침표와 공백으로 문장을 분리
    num_sentences = len(sentences)  # 문장 개수

    # 문장마다 번호 부여 및 리스트에 추가
    numbered_sentences = [f"{i+1}. {sentence}" for i, sentence in enumerate(sentences)]
    numbered_articles.append(numbered_sentences)

# 각 뉴스의 번호가 부여된 문장들을 차례대로 처리
for numbered_article in numbered_articles:
    sentiment_scores = []  # 뉴스의 감성 점수를 저장할 리스트

    # 뉴스의 번호가 부여된 문장들을 순차적으로 처리
    for sentence in numbered_article:
        # senti_classifier를 사용하여 문장의 감성 점수 계산
        score = senti_classifier(sentence)

        if len(score) >= 3:  # 반환값의 길이가 최소한 3 이상이어야 함
            sentiment_scores.append(score)  # 감성 점수 추가
        else:
            print(f"Invalid score format: {score}")  # 올바른 형식의 점수가 아닌 경우 예외 처리

    # 문장별 감성 점수 출력
    for i, sentence_score in enumerate(sentiment_scores):
        print(f"Sentence {i+1}: {sentence_score}")

    # 문장별 감성 점수 종합하여 기사의 최종 점수 계산
    if len(sentiment_scores) > 0:
        # 감성 점수 리스트에서 긍정, 부정, 중립 점수를 추출
        positive_scores = [score[0] for score in sentiment_scores]
        negative_scores = [score[1] for score in sentiment_scores]
        neutral_scores = [score[2] for score in sentiment_scores]

        print(f"Positive scores: {positive_scores}")
        print(f"Negative scores: {negative_scores}")
        print(f"Neutral scores: {neutral_scores}")
    else:
        print("기사에 문장이 없습니다.")

Invalid score format: [{'label': 'positive', 'score': 0.9991828799247742}]
Invalid score format: [{'label': 'positive', 'score': 0.9058384895324707}]
Invalid score format: [{'label': 'positive', 'score': 0.9798312783241272}]
Invalid score format: [{'label': 'positive', 'score': 0.9716472625732422}]
Invalid score format: [{'label': 'positive', 'score': 0.9993860721588135}]
Invalid score format: [{'label': 'positive', 'score': 0.9997544884681702}]
Invalid score format: [{'label': 'positive', 'score': 0.999845027923584}]
Invalid score format: [{'label': 'positive', 'score': 0.9989874958992004}]
Invalid score format: [{'label': 'positive', 'score': 0.9998512268066406}]
Invalid score format: [{'label': 'positive', 'score': 0.9998763799667358}]
Invalid score format: [{'label': 'positive', 'score': 0.6973026990890503}]
Invalid score format: [{'label': 'negative', 'score': 0.9703580141067505}]
Invalid score format: [{'label': 'negative', 'score': 0.9947874546051025}]
Invalid score format: [{'l

RuntimeError: The size of tensor a (1379) must match the size of tensor b (512) at non-singleton dimension 1